In [1]:
%pwd

'C:\\Users\\bjk1y13\\OneDrive - University of Southampton\\MH000_HHFT_flow\\4_Analysis\\flosp\\hhft'

In [2]:
%cd ./../

C:\Users\bjk1y13\OneDrive - University of Southampton\MH000_HHFT_flow\4_Analysis\flosp


In [3]:
%pwd

'C:\\Users\\bjk1y13\\OneDrive - University of Southampton\\MH000_HHFT_flow\\4_Analysis\\flosp'

In [4]:
import flosp

instantiate ioED class

In [5]:
a = flosp.ioED('hhft','./../../3_Data/')

load data

In [6]:
a.load_csv('./../../3_Data/HH_ED_Flow_Study.csv')# ,nrows=1500) # limit number of rows for quick runtime during dev

----------------------------------------
importing ED csv data to RAW dataframe


In [7]:
a._dataRAW.head(2)

,PSEUDONYMISED_PATIENT_ID,AGE_AT_ARRIVAL,GENDER_NATIONAL_CODE,GENDER_NATIONAL_DESCRIPTION,SITE,ARRIVAL_DTTM,ARRIVAL_MODE_NATIONAL_CODE,ARRIVAL_MODE_NATIONAL_DESCRIPTION,ARRIVAL_MODE_SNOMED_CODE,ARRIVAL_MODE_SNOMED_DESCRIPTION,...,ACUITY_SNOMED_CODE,ACUITY_SNOMED_DESCRIPTION,REGISTERED_BY_CODE,CLINICIAN_REGISTERED_BY_NAME,TRIAGE_NURSE_CODE,TRIAGE_NURSE_NAME,ATTENDING_CLINICIAN_CODE,ATTENDING_CLINICIAN_NAME,DISCHARGE_BY_CODE,CLINICIAN_DISCHARGED_BY_NAME
0,489709,43,2.0,Female,BNHH,01/08/2012 00:04,2.0,Other,NaN,NaN,...,NaN,NaN,R47,Mary Gowing,N235,SN Elizabeth Hadley,A569,Dr Mohammed Shahir,A569,Dr Mohammed Shahir
1,653265,44,1.0,Male,BNHH,01/08/2012 00:37,2.0,Other,NaN,NaN,...,NaN,NaN,R47,Mary Gowing,N235,SN Elizabeth Hadley,A569,Dr Mohammed Shahir,A569,Dr Mohammed Shahir


subsampling for quicker inital runtime during dev

a.small_sample()

In [8]:
a._dataRAW.shape

(567635, 103)

column mapping

In [9]:
hh_cols = {
'PSEUDONYMISED_PATIENT_ID':'dept_patid',
'PSEUDONYMISED_PATIENT_ID':'hosp_patid',
'AGE_AT_ARRIVAL':'age',
'GENDER_NATIONAL_DESCRIPTION':'gender',
'SITE':'site',
'ARRIVAL_DTTM':'arrive_datetime',
'ARRIVAL_MODE_NATIONAL_CODE':'arrive_mode',
'INITIAL_ASSESSMENT_DTTM':'first_triage_datetime',
'SEEN_FOR_TREATMENT_DTTM':'first_dr_datetime',
'SPECIALTY_REQUEST_TIME':'first_adm_request_time',
'SPECIALTY_REFERRED_TO_DESCRIPTION':'adm_referral_loc',
'ADMISSION_FLAG':'adm_flag',
'ATTENDANCE_CONCLUSION_DTTM':'depart_datetime',
'STREAM_LOCAL_CODE':'stream'
}

In [10]:
a.map_columns(hh_cols)

----------------------------------------
mapping column names


In [11]:
a._dataRAW.head(2)

,hosp_patid,age,gender,site,arrive_datetime,arrive_mode,first_triage_datetime,first_dr_datetime,first_adm_request_time,adm_referral_loc,adm_flag,depart_datetime,stream
0,489709,43,Female,BNHH,01/08/2012 00:04,2.0,01/08/2012 00:18,01/08/2012 02:24,NaN,NaN,0,01/08/2012 03:44,MIN
1,653265,44,Male,BNHH,01/08/2012 00:37,2.0,01/08/2012 00:53,01/08/2012 02:43,NaN,NaN,0,01/08/2012 04:08,MIN


one datetime needs clenaing in HHFT dataset:

get data out of class

In [12]:
df = a.get_EDraw()

In [13]:
import numpy as np

In [14]:
df[df.first_triage_datetime == '1899-12-30 00:00:00.000000']

,hosp_patid,age,gender,site,arrive_datetime,arrive_mode,first_triage_datetime,first_dr_datetime,first_adm_request_time,adm_referral_loc,adm_flag,depart_datetime,stream
420884,928348,49,Male,RHCH,22/01/2015 10:45,2.0,1899-12-30 00:00:00.000000,22/01/2015 11:57,NaN,NaN,0,22/01/2015 12:45,MIN


In [15]:
index_vals = df[df.first_triage_datetime == '1899-12-30 00:00:00.000000'].first_triage_datetime.index # = np.nan

In [16]:
df.loc[index_vals,'first_triage_datetime'] = np.nan

return dataframe to class now cleaned

In [17]:
a.replace_EDraw(df)

In [18]:
a.convert_cols_datetime("%d/%m/%Y %H:%M")

C:\Users\bjk1y13\OneDrive - University of Southampton\MH000_HHFT_flow\4_Analysis\flosp\flosp\ioED.py:101: UserWarning: Datetime conversion can be problemtic. Make sure you have used the correct datetime string format for each column. You can call this method multiple times with different 'datetime formats' & 'list of columns' to convert if neccessary.
  warnings.warn("Datetime conversion can be problemtic. Make sure you have used the correct datetime string format for each column. You can call this method multiple times with different 'datetime formats' & 'list of columns' to convert if neccessary.")


Converting cols to datetime...(may take some time depedning on size of df)...
arrive_datetime...converting
first_triage_datetime...converting
first_dr_datetime...converting
depart_datetime...converting


In [19]:
a.create_datetime_from_time('first_adm_request_time','arrive_date','first_adm_request_datetime')

C:\Users\bjk1y13\OneDrive - University of Southampton\MH000_HHFT_flow\4_Analysis\flosp\flosp\ioED.py:154: UserWarning: date column passed does not exist. Defaulting to use "arrive_datetime". NOTE: this will generate arrive_date column.
  warnings.warn('date column passed does not exist. Defaulting to use "arrive_datetime". NOTE: this will generate arrive_date column.')


----------------------------------------
Create datetime column from: first_adm_request_time & arrive_date


C:\Users\bjk1y13\OneDrive - University of Southampton\MH000_HHFT_flow\4_Analysis\flosp\flosp\basic_tools.py:131: UserWarning: 13043 patients detected who have -ve wait times. They have probably rolled over midnight, so we add + 1day to the new datetime column created. This assumes < 24hr stays only.



  warnings.warn(str(datetime_values.shape[0]) + ' patients detected who have -ve wait times. They have probably rolled over midnight, so we add + 1day to the new datetime column created. This assumes < 24hr stays only.\n\n\n')


In [20]:
a.run_tests()

--------------------
Finding missing columns...
age_group try using:  use make_age_group_column
arrive_hour try using:  make_callender_columns
arrive_dayofweek try using:  make_callender_columns
arrive_month try using:  make_callender_columns
arrive_dayofweek_name try using:  make_callender_columns
arrive_week try using:  make_callender_columns
depart_method try using:  
depart_hour try using:  make_callender_columns
depart_dayofweek try using:  make_callender_columns
depart_week try using:  make_callender_columns
depart_month try using:  make_callender_columns
depart_dayofweek_name try using:  make_callender_columns
depart_date try using:  make_callender_columns
waiting_time try using:  
breach_flag try using:  
minutes_today try using:  
minutes_tomo try using:  
breach_datetime try using:  
arr_triage_wait try using:  
arr_dr_wait try using:  
arr_adm_req_wait try using:  
adm_req_dep_wait try using:  
dr_adm_req_wait try using:  
dr_dep_wait try using:  
--------------------
Findin

#### run as much automated cleaning as possible using: 

In [21]:
a.autoclean()

----------------------------------------
<function make_callender_columns at 0x0000017864C948C8>
----------------------------------------
<function make_callender_columns at 0x0000017864C948C8>


In [22]:
a.run_tests()

--------------------
Finding missing columns...
depart_method try using:  
minutes_today try using:  
minutes_tomo try using:  
--------------------
Finding columns with wrong datatypes...
Col  arrive_mode                is: float64 . Expected any of:  [<class 'object'>]
Col  adm_flag                   is: int64 . Expected any of:  [<class 'int'>]
Col  stream                     is: object . Expected any of:  [<class 'float'>, <class 'int'>, <class 'numpy.int64'>, <class 'str'>]
Col  depart_dayofweek_name      is: object . Expected any of:  [<class 'str'>]
Col  age_group                  is: category . Expected any of:  [<class 'str'>, 'pandas category type']


#### alternative to autoclean can step through using:

In [23]:
a.make_callender_columns()

----------------------------------------
<function make_callender_columns at 0x0000017864C948C8>
----------------------------------------
<function make_callender_columns at 0x0000017864C948C8>


In [24]:
a.make_wait_columns()

In [25]:
a.make_breach_columns()

In [26]:
a.make_age_group_column()

In [27]:
a.run_tests()

--------------------
Finding missing columns...
depart_method try using:  
minutes_today try using:  
minutes_tomo try using:  
--------------------
Finding columns with wrong datatypes...
Col  arrive_mode                is: float64 . Expected any of:  [<class 'object'>]
Col  adm_flag                   is: int64 . Expected any of:  [<class 'int'>]
Col  stream                     is: object . Expected any of:  [<class 'float'>, <class 'int'>, <class 'numpy.int64'>, <class 'str'>]
Col  depart_dayofweek_name      is: object . Expected any of:  [<class 'str'>]
Col  age_group                  is: category . Expected any of:  [<class 'str'>, 'pandas category type']


In [28]:
a.saveRAWasCLEAN()

----------------------------------------
saved file: ./../../3_Data/processed/hhft/hhftED.pkl


In [29]:
a.saveRAWasRAW()

----------------------------------------
saved file: ./../../3_Data/processed/hhft/RAW/hhftED.pkl


In [30]:
a.loadPKLasRAW()

----------------------------------------
loaded file: ./../../3_Data/processed/hhft/hhftED.pkl


In [31]:
a._dataRAW.shape

(567635, 40)

# Dev

In [33]:
break

SyntaxError: 'break' outside loop (<ipython-input-33-6aaf1f276005>, line 1)

In [ ]:
f1 = {'yes':'no'}

In [ ]:
for i in f1.keys():
    print(i)

In [ ]:
f1[i]

In [ ]:
def clean_time_col_bylen(df, col):
    #### get mode string length
    mode_len = df[col].dropna().apply(lambda x : len(x)).mode().values[0]
    #### get all cases of non-mode col length
    
    index_toclean = df[col].apply(lambda x : len(x)) != mode_len#.index
    return index_toclean

clean_time_col_bylen(a._dataRAW,'first_triage_datetime')
    